In [1]:
import pandas as pd
import json
from statistics import mean
import numpy as np
from pprint import pprint 
from xgboost import XGBRegressor
import copy 


### Load Data & Clean Data

In [2]:
data_source_file = "./data-frame-friendly.json"

f = open (data_source_file, "r")
data = json.load(f)
df1 = pd.DataFrame.from_dict(data, orient='index')
df1=df1.reset_index()

In [3]:
full_dict_storage = [] 
#0 player did not play 
#1 player is loser 
#2 player is winner
for index,row in df1.iterrows(): 
    
    player_names ={
    "andy":0,
    "brandon":0,
    "brian":0,
    "cade":0,
    "darwin":0,
    "josh":0,
    "lindsey":0,
    "sequential":0,
    "sophie":0,
    "steve":0,
    "steven":0,
    "sun":0,
    "susi":0,
    "susu":0,
    "tang":0,
    "yang":0,
    "andy_acs":0,
    "brandon_acs":0,
    "brian_acs":0,
    "cade_acs":0,
    "darwin_acs":0,
    "josh_acs":0,
    "lindsey_acs":0,
    "sequential_acs":0,
    "sophie_acs":0,
    "steve_acs":0,
    "steven_acs":0,
    "sun_acs":0,
    "susi_acs":0,
    "susu_acs":0,
    "tang_acs":0,
    "yang_acs":0,
    "score_diff":0}
    
    
    min_team_size = min(len(row["team_a"]),len(row["team_b"]))
#     print(min_team_size)
    if row["score_a"] > row["score_b"]: # this means team a won 

#         print(row["team_a"][0]["player_name"])
        for index_2 in range(0,min_team_size,1): # go from 0 to five #HUGE BUG HERE 
        
            row_data = row["team_a"][index_2]
            #doing team a player assignment
            name = row["team_a"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 2
            
            # doing the other team assignment 
            name = row["team_b"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 1
            
            
            #adding acs data 
            name = row["team_a"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_a"][index_2]["average_combat_score"]
            
            name = row["team_b"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_a"][index_2]["average_combat_score"]


    elif row["score_a"] < row["score_b"]: # this means team b won 
        for index_2 in range(0,min_team_size,1): # go from 0 to five
            name = row["team_a"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 1
            
            # doing the other team assignment 
            name = row["team_b"][index_2]["player_name"]
            player_names[name] = player_names.get(name, 0) + 2
            
            #adding acs data 
            name = row["team_a"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_a"][index_2]["average_combat_score"]
            
            name = row["team_b"][index_2]["player_name"]
            new_dict_label = str(name)+"_acs"
            player_names[new_dict_label] = player_names.get(new_dict_label, 0) + row["team_a"][index_2]["average_combat_score"]

    player_names['score_diff'] = abs(row["score_a"] - row["score_b"]) # i think i take abs value
#     player_names['score_diff'] = row["score_a"] - row["score_b"]
    print(player_names) # UNCOMMENT THIS TO SHOW RESULTS 
    
    full_dict_storage.append(player_names)
#     break

{'andy': 0, 'brandon': 0, 'brian': 1, 'cade': 2, 'darwin': 2, 'josh': 1, 'lindsey': 1, 'sequential': 0, 'sophie': 2, 'steve': 1, 'steven': 1, 'sun': 2, 'susi': 0, 'susu': 0, 'tang': 2, 'yang': 0, 'andy_acs': 0, 'brandon_acs': 0, 'brian_acs': 139, 'cade_acs': 141, 'darwin_acs': 264, 'josh_acs': 141, 'lindsey_acs': 367, 'sequential_acs': 0, 'sophie_acs': 139, 'steve_acs': 264, 'steven_acs': 350, 'sun_acs': 350, 'susi_acs': 0, 'susu_acs': 0, 'tang_acs': 367, 'yang_acs': 0, 'score_diff': 9}
{'andy': 0, 'brandon': 0, 'brian': 1, 'cade': 1, 'darwin': 1, 'josh': 1, 'lindsey': 2, 'sequential': 0, 'sophie': 2, 'steve': 2, 'steven': 2, 'sun': 1, 'susi': 0, 'susu': 0, 'tang': 2, 'yang': 0, 'andy_acs': 0, 'brandon_acs': 0, 'brian_acs': 21, 'cade_acs': 180, 'darwin_acs': 118, 'josh_acs': 194, 'lindsey_acs': 180, 'sequential_acs': 0, 'sophie_acs': 21, 'steve_acs': 118, 'steven_acs': 279, 'sun_acs': 279, 'susi_acs': 0, 'susu_acs': 0, 'tang_acs': 194, 'yang_acs': 0, 'score_diff': 6}
{'andy': 2, 'brand

In [4]:
df_generated = pd.DataFrame.from_dict(full_dict_storage)
df_generated = df_generated.replace(np.nan,0)
df_generated # this is the full dataset used for training 

,andy,brandon,brian,cade,darwin,josh,lindsey,sequential,sophie,steve,...,emo milf,emo milf_acs,jktoki,jktoki_acs,buttman,buttman_acs,dwaguntimes,dwaguntimes_acs,b0ngd0ng,b0ngd0ng_acs
0,0,0,1,2,2,1,1,0,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,1,1,1,1,2,0,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,1,2,0,2,1,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,1,2,2,1,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,2,1,2,1,0,2,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2,0,1,1,2,2,1,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2,0,2,2,2,1,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,0,0,1,2,1,1,2,1,0,...,2.0,239.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,0,0,2,2,2,2,1,1,0,...,1.0,171.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,0,1,2,1,2,2,0,2,0,...,0.0,0.0,1.0,122.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# select the x_train columns 
x_train= df_generated[["andy",
    "brandon",
    "brian",
    "cade",
    "darwin",
    "josh",
    "lindsey",
    "sequential",
    "sophie",
    "steve",
    "steven",
    "sun",
    "susi",
    "susu",
    "tang",
    "yang",
    "andy_acs",
    "brandon_acs",
    "brian_acs",
    "cade_acs",
    "darwin_acs",
    "josh_acs",
    "lindsey_acs",
    "sequential_acs",
    "sophie_acs",
    "steve_acs",
    "steven_acs",
    "sun_acs",
    "susi_acs",
    "susu_acs",
    "tang_acs",
    "yang_acs"]]

# x_train = df_generated.iloc[:,:16]
x_train = x_train.to_numpy()
# move column position  https://www.geeksforgeeks.org/how-to-move-a-column-to-first-position-in-pandas-dataframe/
y_train = df_generated[["score_diff"]]
y_train = y_train.to_numpy()

# y_train

In [6]:
x_train

array([[  0,   0,   1, ...,   0, 367,   0],
       [  0,   0,   1, ...,   0, 194,   0],
       [  2,   0,   1, ...,   0, 151,   0],
       ...,
       [  1,   1,   2, ...,   0, 204,   0],
       [  1,   0,   1, ...,   0, 316,   0],
       [  1,   0,   1, ...,   0, 492,   0]], dtype=int64)

### Create XG Regressor Model 

In [7]:
xgb_model = XGBRegressor()
# model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)# try linear parameter and other stuff 
# paramenters can be better tuned 
xgb_model = XGBRegressor(nthread=-1, seed=500, eval_metric="rmse", n_estimators=500,learning_rate=1)

xgb_model.fit(x_train,y_train)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             eval_metric='rmse', gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=500, n_jobs=8, nthread=-1,
             num_parallel_tree=1, predictor='auto', random_state=500,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=500,
             subsample=1, tree_method='exact', validate_parameters=1, ...)

In [8]:
import itertools
# this is where you input the player names and map 
players = ["andy","brian","sun","darwin","sophie","susi","lindsey","josh","brandon","tang"] # 10 players
map_name = "Ascent"
print(len(players)) # players should be 10 

all_combinations = []
for comb in itertools.combinations(players, 5):
    both_combs = {} 
    
    comb = list(comb) # first combination
    both_combs[1] = comb
    
    # run every possible combination of 10 choose 5 
    other_comb = list(set(players)-set(comb))# other combination 
    both_combs[2] = other_comb
    
#     print(other_comb)
    all_combinations.append(both_combs) 
# all_combinations

10


### Predict player ACS (based on average of 3 maps + average of 3 recent games) 

In [9]:
 #https://www.geeksforgeeks.org/read-json-file-using-python/
# data_source_file = "./data copy.json"
data_source_file = "./data-frame-friendly.json"

f = open (data_source_file, "r")
data = json.load(f)
df1 = pd.DataFrame.from_dict(data, orient='index')
df1=df1.reset_index()

### we flatten the df here 
row_storage = []

for main_index,main_row in df1.iterrows(): 
#     print(main_index)
    meta_data = df1[['time','url','map','score_a','score_b']].iloc[main_index]

    teams = ["team_a","team_b"]

    for team_number in teams:
        df2 = pd.json_normalize(df1[team_number].iloc[main_index]) #take the first row 


        for index,row in df2.iterrows():
            new_row = pd.concat([meta_data, row])
            row_storage.append(new_row)


new_df = pd.DataFrame(row_storage)
new_df

### predict player acs 
def calc_player_acs_3(player_name,map_name): # we are taking the top
#     player_name = "brian"
    # def calc_player_average

    recent3_maps = new_df.loc[(new_df['player_name']==player_name)&(new_df['map']==map_name)]['average_combat_score'][-3:].tolist()
    
    recent3_games = new_df.loc[(new_df['player_name']==player_name)]['average_combat_score'][-3:].tolist()
    
    recent_6 = recent3_maps + recent3_games
    
#     print(recent3_maps,recent3_games)
#     print(recent_6)
    average_acs = mean(recent_6)

    
    return average_acs

test_one = calc_player_acs_3('andy','Ascent')
print(test_one)


284.3333333333333


In [10]:
player_x_predict = {
    "andy":0,
    "brandon":0,
    "brian":0,
    "cade":0,
    "darwin":0,
    "josh":0,
    "lindsey":0,
    "sequential":0,
    "sophie":0,
    "steve":0,
    "steven":0,
    "sun":0,
    "susi":0,
    "susu":0,
    "tang":0,
    "yang":0,
    "andy_acs":0,
    "brandon_acs":0,
    "brian_acs":0,
    "cade_acs":0,
    "darwin_acs":0,
    "josh_acs":0,
    "lindsey_acs":0,
    "sequential_acs":0,
    "sophie_acs":0,
    "steve_acs":0,
    "steven_acs":0,
    "sun_acs":0,
    "susi_acs":0,
    "susu_acs":0,
    "tang_acs":0,
    "yang_acs":0}

for p_name in players: 
    acs_average = calc_player_acs_3(p_name,map_name)

    key_name = p_name + "_acs" # we are "predicting" the player's acs by taking their average 
    player_x_predict[key_name] = player_x_predict.get(key_name, 0) + acs_average
    
    
print(player_x_predict)
        

{'andy': 0, 'brandon': 0, 'brian': 0, 'cade': 0, 'darwin': 0, 'josh': 0, 'lindsey': 0, 'sequential': 0, 'sophie': 0, 'steve': 0, 'steven': 0, 'sun': 0, 'susi': 0, 'susu': 0, 'tang': 0, 'yang': 0, 'andy_acs': 284.3333333333333, 'brandon_acs': 99, 'brian_acs': 145.33333333333334, 'cade_acs': 0, 'darwin_acs': 194.83333333333334, 'josh_acs': 191.83333333333334, 'lindsey_acs': 231.16666666666666, 'sequential_acs': 0, 'sophie_acs': 165.83333333333334, 'steve_acs': 0, 'steven_acs': 0, 'sun_acs': 193.66666666666666, 'susi_acs': 139.83333333333334, 'susu_acs': 0, 'tang_acs': 277.3333333333333, 'yang_acs': 0}


### Predict score difference of every combination of players in 2 teams (10 choose 5)

In [11]:
all_combs_storage =[]

# this step is creating each x_train from each combintation 
for i in all_combinations:
    
    player_names = copy.deepcopy(player_x_predict) #need this to copy the acs per person
#     print(player_names)
#     print(i[1])
    for name in i[1]: # person who is 2 matters more 
#         print(name)
        player_names[name] = player_names.get(name, 0) + 1 #assign to team 2 
#     print(player_names)

    for name in i[2]: 
        player_names[name] = player_names.get(name, 0) + 2 #assign to team1 
        
#     print(player_names)
    
    y_data= list(player_names.values())
#     print(y_data)
    y_predict = xgb_model.predict([y_data])
#     print(player_names.keys(),y_predict)
    y_predict = float(y_predict)
    answer = {} 
    answer[y_predict]= player_names.items()
    all_combs_storage.append(answer)
    
pprint(all_combs_storage)
    

[{4.824632167816162: dict_items([('andy', 1), ('brandon', 2), ('brian', 1), ('cade', 0), ('darwin', 1), ('josh', 2), ('lindsey', 2), ('sequential', 0), ('sophie', 1), ('steve', 0), ('steven', 0), ('sun', 1), ('susi', 2), ('susu', 0), ('tang', 2), ('yang', 0), ('andy_acs', 284.3333333333333), ('brandon_acs', 99), ('brian_acs', 145.33333333333334), ('cade_acs', 0), ('darwin_acs', 194.83333333333334), ('josh_acs', 191.83333333333334), ('lindsey_acs', 231.16666666666666), ('sequential_acs', 0), ('sophie_acs', 165.83333333333334), ('steve_acs', 0), ('steven_acs', 0), ('sun_acs', 193.66666666666666), ('susi_acs', 139.83333333333334), ('susu_acs', 0), ('tang_acs', 277.3333333333333), ('yang_acs', 0)])},
 {2.2266623973846436: dict_items([('andy', 1), ('brandon', 2), ('brian', 1), ('cade', 0), ('darwin', 1), ('josh', 2), ('lindsey', 2), ('sequential', 0), ('sophie', 2), ('steve', 0), ('steven', 0), ('sun', 1), ('susi', 1), ('susu', 0), ('tang', 2), ('yang', 0), ('andy_acs', 284.3333333333333), 

### Find the smallest predicted score differences in all combinations

In [13]:
final_answer = [] 
smallest_value = 99
for i in all_combs_storage:
    for k, v in i.items():

        if float(k) <smallest_value: 
            smallest_value = float(k)
#             print('\n',i)
            final_answer = i  
#         break

# print(final_answer)

# print the teams 

team_1 = [] 
team_2 = [] 
for key,values in final_answer.items(): 
#     print(values)
    counter = 0 # stop after 10 
    for each_tuple in values:
#         print(each_tuple[1])
        
        if counter <= 17: # i think there are 17 players?  
            if each_tuple[1] == 1: 
                team_1.append(each_tuple[0])
            elif each_tuple[1] == 2: 
                team_2.append(each_tuple[0])

            counter = counter + 1 
            
            
print(team_1)
print(team_2)

['andy', 'brandon', 'lindsey', 'sun', 'susi']
['brian', 'darwin', 'josh', 'sophie', 'tang']
